Analysing with Decision tree
Setup: put it in the same folder as "unifed_csv_without_duplicated_company.csv" and "ROI.csv" (generated by get_roi notebook) 

In [31]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split   
from sklearn.tree import DecisionTreeClassifier, export_graphviz
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, VotingClassifier 
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from IPython.display import Image
import graphviz
from sklearn.model_selection import GridSearchCV
from statistics import mean

pd.set_option('display.max_columns', None)
# Optional
# import library for visuaiization
from sklearn import tree
import matplotlib.pyplot as plt
from collections import Counter

In [2]:
#Read CSV
df=pd.read_csv("unified_csv.csv")
df.info(max_cols=1000)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4050 entries, 0 to 4049
Data columns (total 96 columns):
 #   Column                                       Non-Null Count  Dtype  
---  ------                                       --------------  -----  
 0   company_name                                 4050 non-null   object 
 1   status                                       4050 non-null   object 
 2   category_group_list                          4050 non-null   object 
 3   num_funding_rounds                           4050 non-null   float64
 4   total_funding_usd                            4050 non-null   float64
 5   founded_on                                   4050 non-null   float64
 6   employee_count                               4050 non-null   object 
 7   cat_commerce_shopping                        4050 non-null   float64
 8   cat_fin_services                             4050 non-null   float64
 9   cat_lending_invests                          4050 non-null   float64
 10  

In [24]:
#Preparing xlabel and ylabel
col= df.columns[:28]
features=df[col]
col= df.columns[42:]
dummy_df=df[col]
features= features.join(dummy_df)
features= features.drop(columns=["has_ipo","has_acquisitions","status_closed", "status_exited", "status_operating","founded_on","founded_on","employee_count" , "company_name", "status", "category_group_list", "total_funding_usd", "average_change_in_funds", "average_momentum", "latest_momentum" ])


col= df.columns[88:]
ylabel=df[col]
ylabel= ylabel.drop(columns=["has_acquisitions"])


ylabel

,status_operating,status_exited,status_closed,has_ipo,average_change_in_funds,average_momentum,latest_momentum
0,1,0,0,0,1.478261,2.964229,2.964229
1,0,1,0,0,0.557546,0.934681,0.331667
2,1,0,0,0,0.579845,0.825104,1.801594
3,1,0,0,0,0.515446,0.228600,0.228600
4,1,0,0,0,0.908425,0.341418,0.298739
...,...,...,...,...,...,...,...
4045,1,0,0,0,1.009921,1.008996,0.834738
4046,1,0,0,0,0.498214,1.521582,0.575279
4047,1,0,0,0,0.173378,1.498871,-0.230100
4048,1,0,0,0,0.559524,0.476957,0.457399


In [26]:
#print value_counts of each column of feature dataframe
for i in features.columns: 
    x= features[i].value_counts()
    print("column: ",i,", value is:",x)
    print()

column:  num_funding_rounds , value is: 2.0     1311
3.0     1018
4.0      684
5.0      417
6.0      241
7.0      158
8.0       98
9.0       47
10.0      26
11.0      20
14.0       8
12.0       7
13.0       6
15.0       3
16.0       2
23.0       1
17.0       1
19.0       1
20.0       1
Name: num_funding_rounds, dtype: int64

column:  cat_commerce_shopping , value is: 0.0    3498
1.0     552
Name: cat_commerce_shopping, dtype: int64

column:  cat_fin_services , value is: 1.0    3848
0.0     202
Name: cat_fin_services, dtype: int64

column:  cat_lending_invests , value is: 0.0    3122
1.0     928
Name: cat_lending_invests, dtype: int64

column:  cat_payments , value is: 0.0    2789
1.0    1261
Name: cat_payments, dtype: int64

column:  fd_rd_latest_investment , value is: 1.0    1741
2.0     900
3.0     533
0.0     495
4.0     242
5.0      88
6.0      29
7.0      22
Name: fd_rd_latest_investment, dtype: int64

column:  fd_rd_investment_type_series_unknown , value is: 0.0     3153
1.0     

In [27]:
#print value_counts of each column of ylabel dataframe
for i in ylabel.columns: 
    x= ylabel[i].value_counts()
    print("column: ",i,", value is:",x)
    print()

column:  status_operating , value is: 1    3587
0     463
Name: status_operating, dtype: int64

column:  status_exited , value is: 0    3715
1     335
Name: status_exited, dtype: int64

column:  status_closed , value is: 0    3922
1     128
Name: status_closed, dtype: int64

column:  has_ipo , value is: 0    4025
1      25
Name: has_ipo, dtype: int64

column:  average_change_in_funds , value is:  0.666667    38
 1.200000    33
 1.333333    29
 1.000000    24
-0.666667    17
             ..
 0.488295     1
 1.460208     1
 1.838384     1
 0.642102     1
 0.394340     1
Name: average_change_in_funds, Length: 3480, dtype: int64

column:  average_momentum , value is: -0.333333    5
 2.380480    3
 0.111111    2
 0.811275    2
 0.760427    2
            ..
-0.239214    1
 0.508306    1
 1.116198    1
-0.055383    1
 1.250000    1
Name: average_momentum, Length: 4015, dtype: int64

column:  latest_momentum , value is:  0.000000    44
 1.076923     6
 0.857143     5
 2.380480     5
 1.200000 

In [28]:
###### DO qcut for all ylabels ####### 

binary_labels = [0,1]

ylabel["average_change_in_funds"]= pd.qcut(ylabel["average_change_in_funds"], 2, labels = binary_labels)
ylabel["average_momentum"]= pd.qcut(ylabel["average_momentum"], 2, labels = binary_labels)
ylabel["latest_momentum"]= pd.qcut(ylabel["latest_momentum"], 2, labels = binary_labels)

ylabel

,status_operating,status_exited,status_closed,has_ipo,average_change_in_funds,average_momentum,latest_momentum
0,1,0,0,0,1,1,1
1,0,1,0,0,0,1,0
2,1,0,0,0,0,1,1
3,1,0,0,0,0,0,0
4,1,0,0,0,1,0,0
...,...,...,...,...,...,...,...
4045,1,0,0,0,1,1,1
4046,1,0,0,0,0,1,0
4047,1,0,0,0,0,1,0
4048,1,0,0,0,0,0,0


In [29]:
#Train model to predict each ylabel
im_arr=[]
im_num_arr=[]

for i in ylabel: 
    X_train, X_test, y_train, y_test = train_test_split(features, ylabel[i], test_size=0.3, random_state=5)
    model=DecisionTreeClassifier()
    model.fit(X_train, y_train)
    
    # use the trained model to predict test set for evaluation
    pred_test = model.predict(X_test)

    # print out evaluation result
    print("Predicting: ", i)
    
    print("Accuracy:{}".format(accuracy_score(y_test, pred_test, normalize=True, sample_weight=None)))
    print("Classification Report:\n{}".format(classification_report(y_test, pred_test)))
    print("Confusion Matrix:\n{}".format(confusion_matrix(y_test, pred_test)))
    print("--------------------------------------------------------------------------")
    
    x_col= features.columns
    col=[]
    num=[]
    importance= model.feature_importances_

    for i in range(len(x_col)):
        str= x_col[i], importance[i]
        col.append(str )
        num.append(importance[i])
    im_arr.append(col)
    im_num_arr.append(num)

Predicting:  status_operating
Accuracy:0.8065843621399177
Classification Report:
              precision    recall  f1-score   support

           0       0.15      0.14      0.15       139
           1       0.89      0.89      0.89      1076

    accuracy                           0.81      1215
   macro avg       0.52      0.52      0.52      1215
weighted avg       0.80      0.81      0.81      1215

Confusion Matrix:
[[ 20 119]
 [116 960]]
--------------------------------------------------------------------------
Predicting:  status_exited
Accuracy:0.8609053497942387
Classification Report:
              precision    recall  f1-score   support

           0       0.92      0.93      0.92      1116
           1       0.12      0.11      0.12        99

    accuracy                           0.86      1215
   macro avg       0.52      0.52      0.52      1215
weighted avg       0.86      0.86      0.86      1215

Confusion Matrix:
[[1035   81]
 [  88   11]]
--------------------------

In [42]:
# Choosing the most important feature (Alter "num_of_top" to get number of top features)

avg_arr= []

for i in range(len(im_num_arr[0])):
    dummy_list=[]
    for j in range(len(im_num_arr)): 
        dummy_list.append(im_num_arr[j][i])
    avg_arr.append(mean(dummy_list))

num_of_top=20

top_index= sorted(range(len(avg_arr)), key=lambda i: avg_arr[i], reverse=True)[:num_of_top]

print("Top ", num_of_top  , " features are: ")

for i in range(len(top_index)): 
    print(features.columns[top_index[i]])

[0, 5, 6, 3, 8, 4, 1, 10, 7, 49, 11, 9, 50, 53, 16, 31, 28, 12, 14, 13]

In [ ]:
# # Create a decision tree 

# dotfile = open("dtree2.dot", 'w')
# tree.export_graphviz(model, out_file = dotfile, feature_names = x.columns)
# dotfile.close()